<a href="https://colab.research.google.com/github/yunhui666/NLP_TripAdvisor_Review_Scrap/blob/main/Tripadvisor_Hotel_Review_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# initialize our browser
# need to change to your own path
path = "/Users/siang-linghsu/opt/anaconda3/share/jupyter/lab/chromedriver"


browser = webdriver.Chrome(path)

# go the specific hotel page

link = "https://www.tripadvisor.in/Hotel_Review-g60763-d93437-Reviews-Hotel_Edison-New_York_City_New_York.html"
browser.get(link)

time.sleep(2)


In [ ]:
# a helper function to check if the button exists on the page
def check_exists_by_xpath(xpath):
    try:
        browser.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [ ]:
# let's make it a helper function too
def click_read_more():
    read_more_exists = check_exists_by_xpath('//span[@class="ui_icon caret-down zedZf"]')
   
    if read_more_exists:
        element = browser.find_element_by_xpath('//span[@class="ui_icon caret-down zedZf"]')
        browser.execute_script("arguments[0].click();", element)
        time.sleep(5)

In [ ]:
# back to the first page:
browser.find_element_by_xpath('//*[@id="component_15"]/div/div[3]/div/div[8]/div/div/a[1]').click()


In [ ]:
#change page number
page_num = 10

reviews_title = []
reviews_text = []
ratings = []
authors= []
stay_dates= []

for i in range(0, page_num):
    
    # expand the reviews
    click_read_more() 
    
    
    # parse to a soup
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    reviews_content = soup.find_all('div', class_='cWwQK MC R2 Gi z Z BB dXjiy')
    time.sleep(3)
    # find all find every reviews
    
    # extract the author, review_text and rating 
    for review in reviews_content:
        
        author= review.find('div',class_ = 'bcaHz').a.text
        review_title = review.find('div',class_ = 'fpMxB MC _S b S6 H5 _a').text
        review_text = review.find('div', class_='pIRBV _T').text
        rating = review.find('div', class_='emWez F1').span['class'][1][-2]
        stay_date = review.find('span',class_='euPKI _R Me S4 H3').text
        # append to our accumulative lists
        reviews_title.append(review_title)
        reviews_text.append(review_text)
        ratings.append(rating)
        authors.append(author)
        stay_dates.append(stay_date[14:])
        
    # use selenium to go to the next page
    if (check_exists_by_xpath('//a[@class="ui_button nav next primary "]')):
        browser.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
        time.sleep(2)


In [ ]:
# to check the length of each attritube

print(len(reviews_title))
print(len(reviews_text))
print(len(ratings))
print(len(authors))
print(len(stay_dates))




50
50
50
50
50


In [ ]:
# store the data into a dataset
data = {
    'reviews_title':reviews_title,
    'reviews_text' : reviews_text,
    'ratings' :ratings,
    'author':authors,
    'stay_date':stay_dates
}

df = pd.DataFrame(data)

# check the shape of our dataframe
df.shape


(50, 5)

In [ ]:
df.head()

,reviews_title,reviews_text,ratings,author,stay_date
0,"Hotel Edison, NYC",I would like to mention Rommel Gopez. I have s...,5,Gerry E,December 2021
1,Fantastic hotel in Times Square,This was our 2nd stay at The Edison and we wil...,5,Ian R,March 2020
2,Another great stay at Hotel Edison!,Our group of friends enjoyed our recent visit ...,5,Jean T,December 2021
3,Girls trip in NYC,"The staff at Hotel Edison were amazing, from t...",5,KimC,December 2021
4,Scott's awesome team!,Big shout out from 4 ladies from Virginia! Gr...,5,Martha M,December 2021


In [ ]:
# write to a csv file
# need to change file name
df.to_csv('hotel1.csv')